# Part 1 – Data Wrangling

In [1]:
import pandas as pd
# Load the datasets
hcp_demos_df = pd.read_csv('hcp-demos.csv')
hcp_personality_df = pd.read_csv('hcp-personality.csv')
hcp_cognitive_df = pd.read_csv('hcp-cognitive.csv')

In [2]:
hcp_demos_df.head()

,Subject,M_22-25,M_31-35,F_31-35,M_26-30,F_26-30,F_22-25,M_36+,F_36+
0,ID986,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID407,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID280,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID1024,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ID1001,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN


In [3]:
#Demographic Dataset
final_hcp_demos_df = pd.DataFrame(columns=['Subject', 'Gender', 'Age'])
for index, row in hcp_demos_df.iterrows():
    subject = row['Subject']
    age_gender_data = row.drop('Subject')
    non_null_columns = age_gender_data.dropna().index
    if non_null_columns.size > 0:
        age_gender_col = non_null_columns[0]
        gender = age_gender_col[0]
        age = age_gender_col[2:]
        final_hcp_demos_df = final_hcp_demos_df.append({
            'Subject': subject, 
            'Gender': gender, 
            'Age': age
        }, ignore_index=True)
final_hcp_demos_df.head()

C:\Users\famuf\AppData\Local\Temp\ipykernel_10924\2742511056.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_hcp_demos_df = final_hcp_demos_df.append({
C:\Users\famuf\AppData\Local\Temp\ipykernel_10924\2742511056.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_hcp_demos_df = final_hcp_demos_df.append({
C:\Users\famuf\AppData\Local\Temp\ipykernel_10924\2742511056.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_hcp_demos_df = final_hcp_demos_df.append({
C:\Users\famuf\AppData\Local\Temp\ipykernel_10924\2742511056.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_hcp_demos_df = final_hcp_demos_df.a

,Subject,Gender,Age
0,ID986,M,22-25
1,ID407,M,22-25
2,ID280,M,22-25
3,ID1024,M,31-35
4,ID1001,F,31-35


In [4]:
hcp_cognitive_df.head()

,Subject,groupA.CardSort_AgeAdj,groupA.PMAT24_A_CR,groupA.ProcSpeed_AgeAdj,groupA.ListSort_AgeAdj,groupB.CardSort_AgeAdj,groupB.PMAT24_A_CR,groupB.ProcSpeed_AgeAdj,groupB.ListSort_AgeAdj
0,ID986,94.70,10.0,101.27,94.33,NaN,NaN,NaN,NaN
1,ID407,NaN,NaN,NaN,NaN,96.21,8.0,62.24,85.26
2,ID280,103.87,21.0,106.90,108.26,NaN,NaN,NaN,NaN
3,ID1024,NaN,NaN,NaN,NaN,114.81,22.0,95.81,107.32
4,ID1001,113.04,19.0,137.66,122.75,NaN,NaN,NaN,NaN


In [5]:
# Cognitive Dataset
cognitive_tasks = {
    'CardSort_AgeAdj': ['groupA.CardSort_AgeAdj', 'groupB.CardSort_AgeAdj'],
    'PMAT24_A_CR': ['groupA.PMAT24_A_CR', 'groupB.PMAT24_A_CR'],
    'ProcSpeed_AgeAdj': ['groupA.ProcSpeed_AgeAdj', 'groupB.ProcSpeed_AgeAdj'],
    'ListSort_AgeAdj': ['groupA.ListSort_AgeAdj', 'groupB.ListSort_AgeAdj']
}
hcp_cognitive_combined = hcp_cognitive_df[['Subject']].copy()
for combined_col, (group_a_col, group_b_col) in cognitive_tasks.items():
    hcp_cognitive_combined[combined_col] = hcp_cognitive_df[group_a_col].fillna(hcp_cognitive_df[group_b_col])
hcp_cognitive_combined.head()


,Subject,CardSort_AgeAdj,PMAT24_A_CR,ProcSpeed_AgeAdj,ListSort_AgeAdj
0,ID986,94.70,10.0,101.27,94.33
1,ID407,96.21,8.0,62.24,85.26
2,ID280,103.87,21.0,106.90,108.26
3,ID1024,114.81,22.0,95.81,107.32
4,ID1001,113.04,19.0,137.66,122.75


In [6]:
hcp_personality_df.head()

,IDNumber,Measure,Score
0,996,LifeSatisf_Unadj,47.8
1,995,LifeSatisf_Unadj,69.0
2,994,LifeSatisf_Unadj,47.9
3,986,LifeSatisf_Unadj,51.0
4,984,LifeSatisf_Unadj,50.3


In [7]:
# Personality Dataset
hcp_personality_df['IDNumber'] = 'ID' + hcp_personality_df['IDNumber'].astype(str)
hcp_personality_pivoted = hcp_personality_df.pivot(index='IDNumber', columns='Measure', values='Score').reset_index()
hcp_personality_pivoted = hcp_personality_pivoted.rename(columns={'IDNumber': 'Subject'})
hcp_personality_pivoted.columns.name = None
hcp_personality_pivoted.head()

,Subject,LifeSatisf_Unadj,MeanPurp_Unadj,NEOFAC_A,NEOFAC_C,NEOFAC_E,NEOFAC_N,NEOFAC_O
0,ID1001,70.1,71.6,42.0,41.0,40.0,3.0,33.0
1,ID1024,42.2,31.5,27.0,27.0,20.0,17.0,39.0
2,ID1040,55.0,63.4,37.0,33.0,31.0,19.0,28.0
3,ID1052,43.0,45.2,30.0,29.0,24.0,23.0,22.0
4,ID1064,59.7,57.4,30.0,29.0,36.0,17.0,27.0


In [8]:
# Merging the three dataframes
final_hcp_demos_df['Subject'] = final_hcp_demos_df['Subject'].astype(str)
hcp_cognitive_combined['Subject'] = hcp_cognitive_combined['Subject'].astype(str)
hcp_personality_pivoted['Subject'] = hcp_personality_pivoted['Subject'].astype(str)
hcp_combined = final_hcp_demos_df.merge(hcp_cognitive_combined, on='Subject', how='inner')
hcp_combined = hcp_combined.merge(hcp_personality_pivoted, on='Subject', how='inner')

hcp_combined.head()

,Subject,Gender,Age,CardSort_AgeAdj,PMAT24_A_CR,ProcSpeed_AgeAdj,ListSort_AgeAdj,LifeSatisf_Unadj,MeanPurp_Unadj,NEOFAC_A,NEOFAC_C,NEOFAC_E,NEOFAC_N,NEOFAC_O
0,ID986,M,22-25,94.70,10.0,101.27,94.33,51.0,40.8,25.0,35.0,38.0,28.0,33.0
1,ID407,M,22-25,96.21,8.0,62.24,85.26,64.8,31.4,19.0,31.0,33.0,18.0,25.0
2,ID280,M,22-25,103.87,21.0,106.90,108.26,51.1,49.9,36.0,33.0,38.0,20.0,36.0
3,ID1024,M,31-35,114.81,22.0,95.81,107.32,42.2,31.5,27.0,27.0,20.0,17.0,39.0
4,ID1001,F,31-35,113.04,19.0,137.66,122.75,70.1,71.6,42.0,41.0,40.0,3.0,33.0
